In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd /content/drive/My Drive/Research Project

/content/drive/My Drive/Research Project


In [1]:
import pandas as pd
import numpy as np
import os
# import src
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# pip install -U mittens

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from statistics import mean, stdev, median, mode
# With PCA
from sklearn.decomposition import PCA

from sklearn.feature_extraction.text import CountVectorizer
# SVM
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from mittens import GloVe, Mittens

In [4]:
# pip install autocorrect

In [5]:

import nltk
import string
from nltk.tokenize import word_tokenize
import random
import pickle
from nltk.corpus import stopwords

from autocorrect import Speller
# from pycontractions import Contractions

# from spellchecker import SpellChecker

import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 

from hyperopt import fmin, tpe, hp

In [6]:
# nltk.download('all')

In [7]:
from numpy import asarray
from numpy import zeros
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [8]:
# nltk.download('all')

In [9]:
# # Load your favorite word2vec model
# cont = Contractions('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz')
# text = "we're"
# text = list(cont.expand_texts([text], precise=True))[0]


## Getting Data

In [10]:
# data_path = r'/content/drive/My Drive/Research Project/Data'

In [11]:
data_path = r'C:\Users\kalya\OneDrive - University of Illinois at Chicago\!UIC\!Semesters\3rd Sem\CS 583 Data Mining and Text Mining\Research Project\Data'

In [12]:
# obama_train_pr_df = pd.read_csv(os.path.join(data_path, 'Obama Training1 Data.csv'), usecols = [1,2])
romney_train_pr_df = pd.read_csv(os.path.join(data_path, 'Romney Training1 Data.csv'), usecols = [1,2])

In [13]:
romney_train_pr_df.head()

,Doc Text,Sentiment
0,insidious mitt romney bain help philip morris ...,Negative
1,mean like romney cheat primary,Negative
2,mitt romney still believe black president,Negative
3,romney tax plan deserve nd look secret one dif...,Negative
4,hope romney debate prepped people last time,Positive


In [14]:
# obama_train_pr_df = obama_train_pr_df.dropna()
romney_train_pr_df = romney_train_pr_df.dropna()

In [15]:
romney_train_pr_df.head()

,Doc Text,Sentiment
0,insidious mitt romney bain help philip morris ...,Negative
1,mean like romney cheat primary,Negative
2,mitt romney still believe black president,Negative
3,romney tax plan deserve nd look secret one dif...,Negative
4,hope romney debate prepped people last time,Positive


In [16]:
def tf_idf(x):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(x)
    #print(tfidf_matrix)
    x1 = tfidf_matrix.toarray()
#     print(x1)
    return x1

In [17]:
# obama_train_pr_tfidf = tf_idf(obama_train_pr_df['Doc Text'])
romney_train_pr_tfidf = tf_idf(romney_train_pr_df['Doc Text'])

In [18]:
romney_train_pr_tfidf.shape

(5647, 6544)

### Cross Valid Function

In [19]:
def cross_valid(X,y,model):
    f1_Positive  =[]
    f1_Neutral =[]
    f1_Negative =[]
    acc =[]
    cv = KFold(n_splits=5,shuffle=True)
    for train_index, val_index in cv.split(X):
    #     print("Train Index: ", train_index, "\n")
    #     print("Test Index: ", test_index)
    #     print(X.iloc[train_index])
    #     print(f)
        # print(1)
        X_train1, X_val, y_train1, y_val = X[train_index], X[val_index], y.iloc[train_index], y.iloc[val_index]
        temp = y_train1.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        model = model.fit(X_train1, temp)

        y_pred = model.predict(X_val)
        temp_y_val = y_val.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        f1_temp = f1_score(temp_y_val, y_pred, average = None)

        f1_Positive.append(f1_temp[0])
        f1_Neutral.append(f1_temp[1])
        f1_Negative.append(f1_temp[2])

        acc.append(round(accuracy_score(temp_y_val, y_pred),3))

    return round(mean(acc),3), round(mean(f1_Positive),3), round(mean(f1_Neutral),3), round(mean(f1_Negative),3)

### Cross Valid func with PCA

In [20]:
def cross_valid_PCA(X,y,model, n):
    f1_Positive  =[]
    f1_Neutral =[]
    f1_Negative =[]
    acc =[]
    cv = KFold(n_splits=5,shuffle=True)
    
    pca = PCA(n_components=n)
    
    
    for train_index, val_index in cv.split(X):
    #     print("Train Index: ", train_index, "\n")
    #     print("Test Index: ", test_index)
    #     print(X.iloc[train_index])
    #     print(f)
        # print(1)
        X_train1, X_val, y_train1, y_val = X[train_index], X[val_index], y.iloc[train_index], y.iloc[val_index]
        x_pca1 = pca.fit_transform(X_train1)
        temp = y_train1.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        model = model.fit(x_pca1, temp)
        
        x_pca_val = pca.transform(X_val)
        y_pred = model.predict(x_pca_val)
        temp_y_val = y_val.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        f1_temp = f1_score(temp_y_val, y_pred, average = None)

        f1_Positive.append(f1_temp[0])
        f1_Neutral.append(f1_temp[1])
        f1_Negative.append(f1_temp[2])

        acc.append(round(accuracy_score(temp_y_val, y_pred),3))

    return round(mean(acc),3), round(mean(f1_Positive),3), round(mean(f1_Neutral),3), round(mean(f1_Negative),3)

### Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression

In [25]:


# # clf = SVC(kernel="rbf", gamma=1, class_weight='balanced')
# #     error = 1-(cross_val_score(clf, x_pca1, temp, cv = 5, scoring = 'f1_macro'))
# #     f1 = mean(error) + stdev(error) 
# #print(f1)


# clf = LogisticRegression(class_weight='balanced')
# accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
# eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
# # print([f1_list[2],f1_list[1],f1_list[0]])

# print(eval_dict)


In [31]:
# With Bayesian Optimization, and LR

def objective_func_LR(args):

    C = args['C']
    # penalty = args['penalty']
    # multi_class = args['multi_class']
    # l1_ratio = args['l1_ratio']

 
    clf = LogisticRegression(C= C,class_weight='balanced',n_jobs =-1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(romney_train_pr_tfidf,romney_train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(eval_dict)
    return -(eval_score)


space = {'C': hp.uniform('C', 0,10),
        # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
        # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),

        #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
        }                        
                                
                                
best_classifier_LR = fmin(objective_func_LR, space, algo=tpe.suggest, max_evals=10)
print(best_classifier_LR)

{'accuracy': 0.557, 'f1_pos': 0.479, 'f1_neu': 0.44, 'f1_neg': 0.66, 'eval_score': 0.565}
{'accuracy': 0.548, 'f1_pos': 0.463, 'f1_neu': 0.435, 'f1_neg': 0.651, 'eval_score': 0.554}
{'accuracy': 0.535, 'f1_pos': 0.439, 'f1_neu': 0.422, 'f1_neg': 0.642, 'eval_score': 0.539}
{'accuracy': 0.559, 'f1_pos': 0.494, 'f1_neu': 0.451, 'f1_neg': 0.655, 'eval_score': 0.569}
{'accuracy': 0.557, 'f1_pos': 0.486, 'f1_neu': 0.44, 'f1_neg': 0.658, 'eval_score': 0.567}
{'accuracy': 0.554, 'f1_pos': 0.461, 'f1_neu': 0.449, 'f1_neg': 0.656, 'eval_score': 0.557}
{'accuracy': 0.553, 'f1_pos': 0.498, 'f1_neu': 0.44, 'f1_neg': 0.649, 'eval_score': 0.567}
{'accuracy': 0.55, 'f1_pos': 0.453, 'f1_neu': 0.438, 'f1_neg': 0.656, 'eval_score': 0.553}
{'accuracy': 0.551, 'f1_pos': 0.447, 'f1_neu': 0.435, 'f1_neg': 0.661, 'eval_score': 0.553}
{'accuracy': 0.554, 'f1_pos': 0.477, 'f1_neu': 0.441, 'f1_neg': 0.654, 'eval_score': 0.562}
100%|██████████| 10/10 [23:31<00:00, 141.16s/trial, best loss: -0.569]
{'C': 0.505379

In [27]:
# With PCA, Bayesian Optimization, and LR

def objective_func_LR_PCA(args):

    C = args['C']
    n = args['n']
    
    # penalty = args['penalty']
    # multi_class = args['multi_class']
    # l1_ratio = args['l1_ratio']


    clf = LogisticRegression(C= C,class_weight='balanced',n_jobs =-1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(romney_train_pr_tfidf,romney_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    par_dict = {'C': C, 'n': n}
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(par_dict)
    print(eval_dict)
    print('\n')    
    return -(eval_score)


space = {'C': hp.uniform('C', 0,2),
         'n': hp.choice('n', np.arange(100,500, step =50))
        # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
        # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),
        #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
        }                        
                                
                                
best_classifier_LR_PCA = fmin(objective_func_LR_PCA, space, algo=tpe.suggest, max_evals=10)
print(best_classifier_LR_PCA)

{'C': 0.8571377955328698, 'n': 200}                   
{'accuracy': 0.524, 'f1_pos': 0.479, 'f1_neu': 0.439, 'f1_neg': 0.605, 'eval_score': 0.536}
{'C': 0.40387625537931315, 'n': 100}                                
{'accuracy': 0.504, 'f1_pos': 0.448, 'f1_neu': 0.433, 'f1_neg': 0.581, 'eval_score': 0.511}
{'C': 1.794597882868036, 'n': 300}                                  
{'accuracy': 0.533, 'f1_pos': 0.48, 'f1_neu': 0.442, 'f1_neg': 0.62, 'eval_score': 0.544}
{'C': 0.37922168127727574, 'n': 450}                                
{'accuracy': 0.538, 'f1_pos': 0.49, 'f1_neu': 0.436, 'f1_neg': 0.626, 'eval_score': 0.551}
{'C': 0.39877146758101367, 'n': 300}                                
{'accuracy': 0.528, 'f1_pos': 0.469, 'f1_neu': 0.435, 'f1_neg': 0.62, 'eval_score': 0.539}
{'C': 1.2532023308159683, 'n': 100}                                 
{'accuracy': 0.503, 'f1_pos': 0.45, 'f1_neu': 0.432, 'f1_neg': 0.581, 'eval_score': 0.511}
{'C': 0.23684789267426987, 'n': 400}                 

In [32]:
# # With Bayesian Optimization, different class weights and LR

# def objective_func_LR(args):

#     C = args['C']
#     balance = args['balance']
#     # multi_class = args['multi_class']
#     # l1_ratio = args['l1_ratio']

 
#     clf = LogisticRegression(C= C,class_weight=balance,n_jobs =-1)
#     accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(romney_train_pr_tfidf,romney_train_pr_df['Sentiment'], clf) 
#     eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
#     eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
#     print(eval_dict)
#     return -(eval_score)


# space = {'C': hp.uniform('C', 0,10),
#          'balance': hp.choice('balance', [{1:3, 2:1, 3:2}, {1:15, 2:1, 3:10}, {1:20, 2:5, 3:10}, {1:100, 2:10, 3:100}])
#         # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
#         # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),

#         #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
#         }                        
                                
                                
# best_classifier_LR = fmin(objective_func_LR, space, algo=tpe.suggest, max_evals=20)
# print(best_classifier_LR)

### SVM

In [ ]:
C = 1
n = 3
kernel = 'rbf'

par_dict = {'C': C, 'n': n, 'kernel': kernel}
print(par_dict)

clf = SVC(C= C, kernel = kernel, class_weight='balanced')
accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(romney_train_pr_tfidf,romney_train_pr_df['Sentiment'], clf, n) 
eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
print(eval_dict)
print('\n')    
return -(eval_score)

In [30]:
# With PCA, Bayesian Optimization, and SVM

def objective_func_SVM_PCA(args):
    C = args['C']
    n = args['n']
#     kernel = args['kernel']
#     gamma = args['gamma']
#     degree = args['degree']

    par_dict = {'C': C, 'n': n, 'kernel': 'rbf'}
    print(par_dict)
 
    clf = SVC(C= C, kernel = 'rbf', class_weight='balanced')
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(romney_train_pr_tfidf,romney_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(eval_dict)
    print('\n')    
    return -(eval_score)


space = {'C': hp.uniform('C', 0,1),
         'n': hp.choice('n', np.arange(3, 1003, step =100))
#         'kernel': hp.choice('kernel', ['poly', 'rbf']) 
#         'gamma': hp.choice('gamma',range(1,4)),
#          'degree' : hp.choice('degree',range(1,4))
        }
                                
                       
                                
                                
                                
best_classifier_SVM_PCA = fmin(objective_func_SVM_PCA, space, algo=tpe.suggest, max_evals=20)
print(best_classifier_SVM_PCA)

{'C': 0.6152775265666071, 'n': 603, 'kernel': 'rbf'}  
{'accuracy': 0.548, 'f1_pos': 0.508, 'f1_neu': 0.451, 'f1_neg': 0.632, 'eval_score': 0.563}
{'C': 0.45868449743882456, 'n': 903, 'kernel': 'rbf'}                
{'accuracy': 0.543, 'f1_pos': 0.507, 'f1_neu': 0.462, 'f1_neg': 0.619, 'eval_score': 0.556}
{'C': 0.3693150939132843, 'n': 903, 'kernel': 'rbf'}                 
{'accuracy': 0.538, 'f1_pos': 0.489, 'f1_neu': 0.464, 'f1_neg': 0.615, 'eval_score': 0.547}
{'C': 0.23872776219369496, 'n': 903, 'kernel': 'rbf'}                
{'accuracy': 0.534, 'f1_pos': 0.469, 'f1_neu': 0.473, 'f1_neg': 0.605, 'eval_score': 0.536}
{'C': 0.5200857592502984, 'n': 503, 'kernel': 'rbf'}                 
{'accuracy': 0.546, 'f1_pos': 0.494, 'f1_neu': 0.456, 'f1_neg': 0.633, 'eval_score': 0.558}
{'C': 0.7674861655083285, 'n': 3, 'kernel': 'rbf'}                   
{'accuracy': 0.407, 'f1_pos': 0.26, 'f1_neu': 0.409, 'f1_neg': 0.446, 'eval_score': 0.371}
{'C': 0.22748891813962124, 'n': 503, 'kernel

### KNN

In [31]:
# WIth Bayesian Optimization and KNN
def objective_func_KNN(args):
    n_neighbors = args['n_neighbors']
    metric = args['metric']  

    par_dict = {'n_neighbors': n_neighbors, 'metric': metric}
    print(par_dict)
    
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(romney_train_pr_tfidf,romney_train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_neighbors': hp.choice('n_neighbors',np.arange(1,31, step =3)),
        'metric':hp.choice('metric', ["euclidean","manhattan"])
        }

best_classifier_KNN = fmin(objective_func_KNN, space, algo=tpe.suggest, max_evals=10)
print(best_classifier_KNN)

{'n_neighbors': 1, 'metric': 'manhattan'}             
{'accuracy': 0.454, 'f1_pos': 0.368, 'f1_neu': 0.412, 'f1_neg': 0.53, 'eval_score': 0.451}
{'n_neighbors': 28, 'metric': 'euclidean'}                           
{'accuracy': 0.545, 'f1_pos': 0.407, 'f1_neu': 0.329, 'f1_neg': 0.675, 'eval_score': 0.542}
{'n_neighbors': 25, 'metric': 'manhattan'}                           
{'accuracy': 0.424, 'f1_pos': 0.385, 'f1_neu': 0.357, 'f1_neg': 0.49, 'eval_score': 0.433}
{'n_neighbors': 28, 'metric': 'manhattan'}                           
{'accuracy': 0.44, 'f1_pos': 0.392, 'f1_neu': 0.324, 'f1_neg': 0.519, 'eval_score': 0.45}
{'n_neighbors': 10, 'metric': 'euclidean'}                           
{'accuracy': 0.509, 'f1_pos': 0.414, 'f1_neu': 0.381, 'f1_neg': 0.62, 'eval_score': 0.514}
{'n_neighbors': 28, 'metric': 'euclidean'}                           
{'accuracy': 0.542, 'f1_pos': 0.41, 'f1_neu': 0.315, 'f1_neg': 0.674, 'eval_score': 0.542}
{'n_neighbors': 22, 'metric': 'manhattan'}       

In [32]:
# WIth PCA , Bayesian Optimization and KNN
def objective_func_KNN_PCA(args):
    n_neighbors = args['n_neighbors']
    metric = args['metric']
    n = args['n']    

    par_dict = {'n_neighbors': n_neighbors, 'n': n, 'metric': metric}
    print(par_dict)
    
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(romney_train_pr_tfidf,romney_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_neighbors': hp.choice('n_neighbors',np.arange(1,10, step =1)),
        'metric':hp.choice('metric', ["euclidean","manhattan"]),
        'n': hp.choice('n', np.arange(3,63, step =5))}

best_classifier_KNN_PCA = fmin(objective_func_KNN_PCA, space, algo=tpe.suggest, max_evals=15)
print(best_classifier_KNN_PCA)

{'n_neighbors': 7, 'n': 13, 'metric': 'euclidean'}    
{'accuracy': 0.488, 'f1_pos': 0.297, 'f1_neu': 0.359, 'f1_neg': 0.614, 'eval_score': 0.466}
{'n_neighbors': 5, 'n': 33, 'metric': 'manhattan'}                  
{'accuracy': 0.488, 'f1_pos': 0.354, 'f1_neu': 0.379, 'f1_neg': 0.602, 'eval_score': 0.481}
{'n_neighbors': 6, 'n': 38, 'metric': 'manhattan'}                  
{'accuracy': 0.499, 'f1_pos': 0.341, 'f1_neu': 0.39, 'f1_neg': 0.616, 'eval_score': 0.485}
{'n_neighbors': 1, 'n': 23, 'metric': 'euclidean'}                  
{'accuracy': 0.482, 'f1_pos': 0.345, 'f1_neu': 0.369, 'f1_neg': 0.598, 'eval_score': 0.475}
{'n_neighbors': 4, 'n': 48, 'metric': 'euclidean'}                  
{'accuracy': 0.49, 'f1_pos': 0.371, 'f1_neu': 0.41, 'f1_neg': 0.589, 'eval_score': 0.483}
{'n_neighbors': 2, 'n': 8, 'metric': 'manhattan'}                   
{'accuracy': 0.384, 'f1_pos': 0.332, 'f1_neu': 0.379, 'f1_neg': 0.422, 'eval_score': 0.379}
{'n_neighbors': 3, 'n': 3, 'metric': 'manhattan'}  

### Naive Bayes

In [35]:
clf = GaussianNB()
accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(romney_train_pr_tfidf,romney_train_pr_df['Sentiment'], clf) 
eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
print(eval_dict)

{'accuracy': 0.395, 'f1_pos': 0.364, 'f1_neu': 0.361, 'f1_neg': 0.444, 'eval_score': 0.401}


In [33]:
# With PCA and Naive bayes
def objective_func_NB_PCA(args):
    n = args['n']

    par_dict = {'n': n}
    print(par_dict)
    
    clf = GaussianNB()
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(romney_train_pr_tfidf,romney_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n': hp.choice('n', np.arange(3,63, step =1))}

best_classifier_NB_PCA = fmin(objective_func_NB_PCA, space, algo=tpe.suggest, max_evals=10)



{'n': 35}                                             
{'accuracy': 0.44, 'f1_pos': 0.311, 'f1_neu': 0.422, 'f1_neg': 0.503, 'eval_score': 0.418}
{'n': 56}                                                           
{'accuracy': 0.432, 'f1_pos': 0.328, 'f1_neu': 0.45, 'f1_neg': 0.455, 'eval_score': 0.405}
{'n': 49}                                                           
{'accuracy': 0.433, 'f1_pos': 0.339, 'f1_neu': 0.447, 'f1_neg': 0.459, 'eval_score': 0.41}
{'n': 28}                                                           
{'accuracy': 0.457, 'f1_pos': 0.306, 'f1_neu': 0.392, 'f1_neg': 0.547, 'eval_score': 0.437}
{'n': 62}                                                           
{'accuracy': 0.43, 'f1_pos': 0.322, 'f1_neu': 0.448, 'f1_neg': 0.454, 'eval_score': 0.402}
{'n': 3}                                                            
{'accuracy': 0.498, 'f1_pos': 0.097, 'f1_neu': 0.156, 'f1_neg': 0.656, 'eval_score': 0.417}
{'n': 46}                                           

## Ensemble Methods

### Random Forest

In [34]:
# With Bayesian Optimization, and Random Forest

def objective_func_RF(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']

    par_dict = {'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(romney_train_pr_tfidf,romney_train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
        }
                                
                                
                                
best_classifier_RF = fmin(objective_func_RF, space, algo=tpe.suggest, max_evals=25)
print(best_classifier_RF)

{'n_estimators': 36, 'max_depth': 78, 'min_samples_split': 8, 'min_samples_leaf': 7}
{'accuracy': 0.543, 'f1_pos': 0.194, 'f1_neu': 0.158, 'f1_neg': 0.692, 'eval_score': 0.476}
{'n_estimators': 53, 'max_depth': 22, 'min_samples_split': 3, 'min_samples_leaf': 2}
{'accuracy': 0.525, 'f1_pos': 0.075, 'f1_neu': 0.051, 'f1_neg': 0.683, 'eval_score': 0.428}
{'n_estimators': 54, 'max_depth': 71, 'min_samples_split': 4, 'min_samples_leaf': 3}
{'accuracy': 0.554, 'f1_pos': 0.247, 'f1_neu': 0.195, 'f1_neg': 0.697, 'eval_score': 0.499}
{'n_estimators': 81, 'max_depth': 61, 'min_samples_split': 5, 'min_samples_leaf': 7}
{'accuracy': 0.539, 'f1_pos': 0.173, 'f1_neu': 0.133, 'f1_neg': 0.689, 'eval_score': 0.467}
{'n_estimators': 91, 'max_depth': 38, 'min_samples_split': 11, 'min_samples_leaf': 10}
{'accuracy': 0.528, 'f1_pos': 0.118, 'f1_neu': 0.043, 'f1_neg': 0.685, 'eval_score': 0.444}
{'n_estimators': 55, 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 5}
{'accuracy': 0.533, 'f1_pos'

In [35]:
# With PCA,  Bayesian Optimization, and Random Forest

def objective_func_RF_PCA(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']
    n = args['n']

    par_dict = { 'n': n, 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(romney_train_pr_tfidf,romney_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1)),
        'n': hp.choice('n', np.arange(3,203, step =20))
        }
                                
                                
                                
best_classifier_RF_PCA = fmin(objective_func_RF_PCA, space, algo=tpe.suggest, max_evals=30)
print(best_classifier_RF_PCA)

{'n': 123, 'n_estimators': 54, 'max_depth': 91, 'min_samples_split': 9, 'min_samples_leaf': 1}
{'accuracy': 0.563, 'f1_pos': 0.292, 'f1_neu': 0.308, 'f1_neg': 0.698, 'eval_score': 0.518}
{'n': 123, 'n_estimators': 30, 'max_depth': 87, 'min_samples_split': 7, 'min_samples_leaf': 6}
{'accuracy': 0.554, 'f1_pos': 0.25, 'f1_neu': 0.294, 'f1_neg': 0.695, 'eval_score': 0.5}
{'n': 183, 'n_estimators': 36, 'max_depth': 93, 'min_samples_split': 11, 'min_samples_leaf': 10}
{'accuracy': 0.558, 'f1_pos': 0.21, 'f1_neu': 0.252, 'f1_neg': 0.699, 'eval_score': 0.489}
{'n': 143, 'n_estimators': 42, 'max_depth': 33, 'min_samples_split': 8, 'min_samples_leaf': 7}
{'accuracy': 0.551, 'f1_pos': 0.239, 'f1_neu': 0.247, 'f1_neg': 0.694, 'eval_score': 0.495}
{'n': 63, 'n_estimators': 85, 'max_depth': 99, 'min_samples_split': 4, 'min_samples_leaf': 8}
{'accuracy': 0.564, 'f1_pos': 0.262, 'f1_neu': 0.301, 'f1_neg': 0.699, 'eval_score': 0.508}
{'n': 123, 'n_estimators': 74, 'max_depth': 58, 'min_samples_split':

In [ ]:
# # WITH LDA
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from math import sqrt
# lda = LinearDiscriminantAnalysis(n_components = None)
# x_lda = lda.fit_transform(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'])

In [ ]:
# # With LDA, Bayesian Optimization, and Random Forest

# def objective_func(args):
#     n_estimators = args['n_estimators']
#     max_depth = args['max_depth']
#     min_samples_split = args['min_samples_split']
#     min_samples_leaf = args['min_samples_leaf']
# #     pca = PCA(n_components=args['n'])
# #     x_pca1 = pca.fit_transform(obama_train_pr_tfidf)

 
#     clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    
#     temp = obama_train_pr_df['Sentiment'].apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
#     temp_f1 = cross_val_score(clf, x_lda, temp, cv = 5, scoring = 'f1_macro')
#     f1 = mean(temp_f1)
#     return -(f1)
# space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
#          'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
# #         'criterion': hp.choice('criterion',['gini', 'entropy']),
# #         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
#         'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
#          'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
# #         'n': hp.choice('n', np.arange(3,20, step =1))
#         }
                                
                                
                                
# best_classifier = fmin(objective_func, space, algo=tpe.suggest, max_evals=30)
# print(best_classifier)# With Bayesian Optimization, and Random Forest



In [ ]:
# obama_train2_pr_tfidf, obama_val_pr_tfidf, obama_train2_pr_class, obama_val_pr_class = train_test_split(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'], test_size = 0.2)

In [ ]:
# x_lda2 = lda.fit_transform(obama_train2_pr_tfidf, obama_train2_pr_class)

In [ ]:
# # With LDA, Bayesian Optimization, and Random Forest

# def objective_func(args):
#     n_estimators = args['n_estimators']
#     max_depth = args['max_depth']
#     min_samples_split = args['min_samples_split']
#     min_samples_leaf = args['min_samples_leaf']
# #     pca = PCA(n_components=args['n'])
# #     x_pca1 = pca.fit_transform(obama_train_pr_tfidf)

 
#     clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    
# #     temp = obama_train_pr_df['Sentiment'].apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
#     temp_f1 = cross_val_score(clf, x_lda2, obama_train2_pr_class, cv = 5, scoring = 'f1_macro')
#     f1 = mean(temp_f1)
#     return -(f1)
# space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
#          'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
# #         'criterion': hp.choice('criterion',['gini', 'entropy']),
# #         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
#         'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
#          'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
# #         'n': hp.choice('n', np.arange(3,20, step =1))
#         }
                                
                                
                                
# best_classifier = fmin(objective_func, space, algo=tpe.suggest, max_evals=30)
# print(best_classifier)# With Bayesian Optimization, and Random Forest



In [ ]:
# x_val_lda = lda.transform(obama_val_pr_tfidf)
# x_val_lda_df = pd.DataFrame(data = x_val_lda)

In [ ]:
# c = lambda b : 1 if b == 0 else b
# d = lambda b : 2 if b <= 1 else b
# bc = RandomForestClassifier(max_depth = c(best_classifier['max_depth']), min_samples_leaf = c(best_classifier['min_samples_leaf']), min_samples_split = d(best_classifier['min_samples_split'])
#                             ,n_estimators = best_classifier['n_estimators'])

# bc.fit(x_lda2, obama_train2_pr_class)

# y_pred = bc.predict(x_val_lda)

# accuracy_score(obama_val_pr_class, y_pred)

In [ ]:
# lda_clf = lda.fit(obama_train2_pr_tfidf, obama_train2_pr_class)
# y_pred_lda_clf = lda_clf.predict(obama_val_pr_tfidf)
# # y_pred_lda_clf

# accuracy_score(obama_val_pr_class, y_pred_lda_clf)

### Light Gradient Boosting

In [36]:
import lightgbm as lgbm
from lightgbm.sklearn import LGBMClassifier

In [37]:
# With Bayesian Optimization, and Light GB

def objective_func_LGB(args):
    boosting_type= args['boosting_type']
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    learning_rate = args['learning_rate']
    subsample = args['subsample']
    min_split_gain = args['min_split_gain']

    par_dict = {'boosting_type': boosting_type, 'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate, 'subsample': subsample, 'min_split_gain': min_split_gain}
    print(par_dict)
    
    clf = LGBMClassifier(boosting_type = boosting_type, class_weight  = 'balanced', num_leaves = 2**max_depth, n_estimators = n_estimators, max_depth = max_depth, subsample=subsample, learning_rate=learning_rate, min_split_gain = min_split_gain)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(romney_train_pr_tfidf,romney_train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {
    'boosting_type' : hp.choice('boosting_type', ['gbdt', 'dart', 'goss']),
    'n_estimators': hp.choice('n_estimators', np.arange(50,501, step =1)),
    'learning_rate':  hp.uniform('learning_rate', 0,0.2), 
    'max_depth': hp.choice('max_depth', np.arange(3, 10, step =1)),   
     'subsample': hp.choice('subsample', np.linspace(0.8, 1, num=4)),
    'min_split_gain': hp.choice('min_split_gain' , np.arange(0,6, step =1))}
                                
                                
                                
best_classifier_LGB = fmin(objective_func_LGB, space, algo=tpe.suggest, max_evals=25)
print(best_classifier_LGB)

{'boosting_type': 'gbdt', 'n_estimators': 345, 'max_depth': 6, 'learning_rate': 0.010871082478688177, 'subsample': 0.8666666666666667, 'min_split_gain': 0}
{'accuracy': 0.491, 'f1_pos': 0.414, 'f1_neu': 0.466, 'f1_neg': 0.549, 'eval_score': 0.485}
{'boosting_type': 'dart', 'n_estimators': 116, 'max_depth': 5, 'learning_rate': 0.117138490340471, 'subsample': 0.8666666666666667, 'min_split_gain': 5}
{'accuracy': 0.478, 'f1_pos': 0.411, 'f1_neu': 0.457, 'f1_neg': 0.53, 'eval_score': 0.473}
{'boosting_type': 'goss', 'n_estimators': 315, 'max_depth': 9, 'learning_rate': 0.07149395952029858, 'subsample': 0.9333333333333333, 'min_split_gain': 4}
{'accuracy': 0.498, 'f1_pos': 0.426, 'f1_neu': 0.41, 'f1_neg': 0.591, 'eval_score': 0.505}
{'boosting_type': 'dart', 'n_estimators': 256, 'max_depth': 9, 'learning_rate': 0.04951899427471998, 'subsample': 1.0, 'min_split_gain': 0}
{'accuracy': 0.503, 'f1_pos': 0.427, 'f1_neu': 0.462, 'f1_neg': 0.572, 'eval_score': 0.501}
{'boosting_type': 'goss', 'n_e

In [38]:
# With PCA, Bayesian Optimization, and Light GB

def objective_func_LGB_PCA(args):
    boosting_type= args['boosting_type']
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    learning_rate = args['learning_rate']
#     subsample = args['subsample']
    min_split_gain = args['min_split_gain']
    n = args['n']
    
    par_dict = {'n': n, 'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate, 'subsample': 1, 'min_split_gain': min_split_gain} 
    print(par_dict)
    
    clf = LGBMClassifier(boosting_type = boosting_type, class_weight  = 'balanced', num_leaves = 2**max_depth, n_estimators = n_estimators, max_depth = max_depth, subsample=1, learning_rate=learning_rate, min_split_gain = min_split_gain)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(romney_train_pr_tfidf, romney_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {
    'boosting_type' : hp.choice('boosting_type', ['gbdt', 'dart', 'goss']),
    'n_estimators': hp.choice('n_estimators', np.arange(50,501, step =1)),
    'learning_rate':  hp.uniform('learning_rate', 0,0.2), 
    'max_depth': hp.choice('max_depth', np.arange(3, 10, step =1)),   
#      'subsample': hp.choice('subsample', np.linspace(0.8, 1, num=4)),
    'min_split_gain': hp.choice('min_split_gain' , np.arange(0,6, step =1)),
        'n': hp.choice('n', np.arange(3,203, step =20))}
                                
                                
                                
best_classifier_LGB_PCA = fmin(objective_func_LGB_PCA, space, algo=tpe.suggest, max_evals=35)
print(best_classifier_LGB_PCA)

{'n': 43, 'n_estimators': 212, 'max_depth': 6, 'learning_rate': 0.01800028418964581, 'subsample': 1, 'min_split_gain': 1}
{'accuracy': 0.497, 'f1_pos': 0.421, 'f1_neu': 0.429, 'f1_neg': 0.58, 'eval_score': 0.499}
{'n': 163, 'n_estimators': 341, 'max_depth': 6, 'learning_rate': 0.05778800188029725, 'subsample': 1, 'min_split_gain': 3}
{'accuracy': 0.53, 'f1_pos': 0.463, 'f1_neu': 0.446, 'f1_neg': 0.615, 'eval_score': 0.536}
{'n': 83, 'n_estimators': 438, 'max_depth': 5, 'learning_rate': 0.09849562123511453, 'subsample': 1, 'min_split_gain': 1}
{'accuracy': 0.534, 'f1_pos': 0.392, 'f1_neu': 0.404, 'f1_neg': 0.652, 'eval_score': 0.526}
{'n': 163, 'n_estimators': 203, 'max_depth': 9, 'learning_rate': 0.09458342802423055, 'subsample': 1, 'min_split_gain': 4}
{'accuracy': 0.538, 'f1_pos': 0.423, 'f1_neu': 0.406, 'f1_neg': 0.65, 'eval_score': 0.537}
{'n': 183, 'n_estimators': 321, 'max_depth': 6, 'learning_rate': 0.05716678656047405, 'subsample': 1, 'min_split_gain': 1}
{'accuracy': 0.554, 'f

In [39]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(sampling_strategy='not majority',random_state=42)
romney_train_pr_tfidf_smote, romney_train_pr_class = sm.fit_sample(romney_train_pr_tfidf, romney_train_pr_df['Sentiment'] )


sm = SMOTE(sampling_strategy='not majority',random_state=42)
obama_train_pr_tfidf_smote, obama_train_pr_class = sm.fit_sample(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'])

KeyboardInterrupt: 